### 示例：基于SVM的数字识别
（1）收集数据：提供的文本文件。  
（2）准备数据：基于二值图像构造向量。  
（3）分析数据：对图像向量进行目测。  
（4）训练算法：采用两种不同的核函数，并对径向基核函数采用不同的设置来运行SMO算法。  
（5）测试算法：编写一个函数来测试不同的核函数并计算错误率。  
（6）使用算法：一个图像识别的完整应用还需要一些图像处理的知识，本书没有深入介绍。  

In [1]:
import numpy as np

In [2]:
"""
函数说明:将图像格式化处理为一个向量。将一个32*32的二进制图像矩阵转换为1*1024的向量。

Parameters:
    filename - 文件名
Returns:
    returnVector - 1*1024的向量
Modify:
    2018-10-02
"""
def img2vector(filename):
    #返回的numpy矩阵，1*1024
    returnVector = np.zeros((1,1024))
    #打开文件
    fr = open(filename)
    for i in range(32):
          lineStr = fr.readline()
          for j in range(32):
              returnVector[0,32*i+j] = int(lineStr[j])
    return returnVector

In [3]:
"""
函数说明:加载图像数据

Parameters:
    dirName - 数据存放路径
Returns:
    trainingMat - 数据矩阵
    hwLabels - 数据标签
"""
def loadImages(dirName):
    from os import listdir
    hwLabels = []                           #测试集的labels
    trainingFileList = listdir(dirName)     #返回trainingDigits目录下的文件名列表
    m = len(trainingFileList) #返回trainingDigits下的文件个数
    trainingMat = np.zeros((m,1024))#初始化训练的Mat矩阵,测试集
    #从文件名中解析出训练集的类别
    for i in range(m):
        fileNameStr = trainingFileList[i]    #取文件名
        fileStr = fileNameStr.split('.')[0]    #去掉文件名的后缀
        classNumStr = int(fileStr.split('_')[0])    #获取类别
        if classNumStr == 9: hwLabels.append(-1)    #若为9，lable 为-1；其他则为1
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s'%(dirName,fileNameStr))    #将每一个文件转换成1*1024的向量并存到Mat矩阵中
    return trainingMat, hwLabels

In [4]:
class optStruct:
    """
        数据结构，维护所有需要操作的值
        Parameters：
            dataMatIn - 数据矩阵
            classLabels - 数据标签
            C - 松弛变量
            toler - 容错率
            kTup - 核函数信息
        """
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn              #数据矩阵
        self.labelMat = classLabels     #数据标签
        self.C = C                      #惩罚参数
        self.tol = toler                #容错率
        self.m = np.shape(dataMatIn)[0] #样本数
        self.alphas = np.mat(np.zeros((self.m,1)))      #初始化alphas
        self.b = 0      #初始化b
        # 根据矩阵行数初始化误差缓存，第一列为是否有效的标志位，第二列为实际的误差E的值。
        self.eCache = np.mat(np.zeros((self.m,2)))
        #核函数填充
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:,i] = kerenelTrans(self.X, self.X[i,:],kTup)

In [5]:
"""
函数说明:SMO辅助函数

Parameters:
    i - 第一个alpha的下标
    m - 所有alpha的数目
Returns:
    j - 不等于i的下标
"""
def selectJrand(i,m):
    j = i
    while(j == i):
        j = int(np.random.uniform(0,m))
    return j        #选择一个不等于i的j返回

"""
函数说明:SMO辅助函数
调整大于H或小于L的alpha的值
Parameters:
    aj - alpha的值
    H - 限定alpha的最大值
    L - 限定alpha的最小值
Returns:
    aj - 范围限定于[L,H]间的alpha的值
"""
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

"""
函数说明:计算E值

Parameters:
    oS - 数据结构
    k - 标号为k的数据
Returns:
    Ek - 下标为k的数据误差
"""
def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas,oS.labelMat).T*oS.K[:, k] +oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

In [6]:
"""
函数说明：内循环启发方式2
Parameters：
    i - 标号为i的数据的索引值
    oS - 数据结构
    Ei - 标号为i的数据误差
Returns:
    j, maxK - 标号为j或maxK的数据的索引值
    Ej - 标号为j的数据误差
"""
def selectJ(i, oS, Ei):
    maxK = -1; maxDeltaE = 0; Ej = 0        #初始化
    oS.eCache[i] = [1, Ei]                  #根据Ei更新误差缓存
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]   #返回误差不为0的数据的索引值，对一个矩阵.A转换为Array类型
    if(len(validEcacheList)) >1:            #有不为0的误差
        for k in validEcacheList:           #遍历,找到最大的Ek
            if k == i: continue             #不计算i,浪费时间
            Ek = calcEk(oS, k)              #计算Ek
            deltaE = abs(Ei - Ek)           #计算|Ei-Ek|
            if(deltaE > maxDeltaE):         #找到maxDeltaE
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej                     #返回maxK,Ej
    else:                                   #没有不为0的误差
        j = selectJrand(i,oS.m)             #随机选择alpha_j的索引值
        Ej = calcEk(oS,j)                   #计算Ej
    return j,Ej                             #返回j,Ej

"""
函数说明：计算Ek,并更新误差缓存
Parameters：
    oS - 数据结构
    k - 标号为k的数据的索引值
Returns:
    无
"""
def updateEk(oS, k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]

"""
函数说明：完整Platt SMO算法中的优化例程
Parameters：
    i - 标号为i的数据的索引值
    oS - 数据结构
Returns:
    1 - 有任意一对alpha值发生变化
    0 - 没有任意一对alpha值发生变化或变化太小
"""
def innerL(i,oS):
    # 步骤1：计算误差  Ei = f(xi)-yi
    Ei = calcEk(oS,i)
    # 优化alpha，更设定一定的容错率
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        # 随机选择另一个与alpha_i成对优化的alpha_j
        # 步骤1：计算误差Ej Ej = f(xj)-yj
        j,Ej = selectJ(i,oS,Ei)
        # 保存更新前的aplpha值，使用深拷贝
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        # 步骤2：计算L和H
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H: print("L==H"); return 0
        # 步骤3：计算eta
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]
        if eta >= 0: print("eta>=0");return 0
        # 步骤4：更新alpha_j
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        # 步骤5：修剪alpha_j
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print("j not moving enough");return 0
        # 步骤6：更新alpha_i
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        # 步骤7：更新b1和b2
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, i] \
             - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i, j]  \
             - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[j, j]
        # 根据b1和b2确定b
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else: return 0

In [7]:
"""
函数说明：完整Platt SMO算法中的外循环代码
Parameters：
    dataMatIn - 数据矩阵
    classLabels - 数据标签
    C - 惩罚系数
    toler - 容错率
    maxIter - 最大迭代次数
    
Returns:
    oS.b - 参数b
    oS.alphas - 参数alpha
"""
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin',0)):
   oS = optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler,kTup)        #数据结构体初始化
   iter = 0     #迭代次数
   entireSet = True; alphaPairsChanged = 0      #是否遍历整个数据集；参数调整次数
   # 循环跳出条件：1.迭代次数超过最大迭代次数；2.遍历了整个数据集都未对任意的alpha对进行修改
   while(iter < maxIter) and ((alphaPairsChanged >0) or (entireSet)):
       alphaPairsChanged = 0
       if entireSet:
           for i in range(oS.m):
               alphaPairsChanged += innerL(i,oS)
               print("fullSet, iter: %d  i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
           iter += 1
       else:
           nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A <C))[0]  #非边界循环
           for i in nonBoundIs:
               alphaPairsChanged += innerL(i,oS)
               print("non-bound, iter: %d   i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
           iter += 1
       if entireSet:
            entireSet = False
       elif(alphaPairsChanged==0):
            entireSet=True
       print("iteration number: %d" % iter)
   return oS.b, oS.alphas

"""
函数说明：计算w的值
Parameters：
    alphas - 计算好的alpha系数
    dataMatIn - 数据矩阵
    classLabels - 数据标签

Returns:
    w - 参数w
"""
def calcWs(alphas, dataArr, classLabels):
    X = np.mat(dataArr); labelMat = np.mat(classLabels).transpose()
    m,n = np.shape(X)
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [8]:
"""
函数说明：核转换函数
Parameters：
    X - 数据矩阵
    A - 核函数所需参数
    kTup - 核函数信息

Returns:
    K - 核函数计算结果
"""
def kerenelTrans(X, A, kTup):
    m,n = np.shape(X)               #返回数据矩阵的大小
    K = np.mat(np.zeros((m,1)))     #初始化K矩阵
    if kTup[0] == 'lin': K = X*A.T
    elif kTup[0]=='rbf':            #径向基函数（radial bias function）
        for j in range(m):
            deltaRow = X[j,:]-A
            K[j] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2))
    else: raise NameError('Houston We Have a Problem--That Kernel is not recognized')
    return K

In [9]:
"""
函数说明：基于SVM的数字识别
Parameters：
    kTup - 核函数信息

Returns:
    无
"""
def testDigits(kTup = ('rbf', 10)):
    dataArr,labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr,labelArr,200, 0.0001,10000,kTup)
    dataMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A >0)[0]
    sVs = dataMat[svInd]
    labelSV = labelMat[svInd]
    print("there are %d Support Vectors" % np.shape(sVs)[0])
    m,n = np.shape(dataMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kerenelTrans(sVs, dataMat[i,:],kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd])+b
        if np.sign(predict) != np.sign(labelArr[i]): errorCount +=1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr, labelArr = loadImages('testDigits')
    errorCount = 0
    dataMat = np.mat(dataArr);labelMat = np.mat(labelArr).transpose()
    m,n = np.shape(dataMat)
    for i in range(m):
        kernelEval = kerenelTrans(sVs, dataMat[i, :], kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]): errorCount += 1
    print("the test error rate is: %f" % (float(errorCount) / m))

In [10]:
testDigits()

L==H
fullSet, iter: 0  i:0, pairs changed 0
fullSet, iter: 0  i:1, pairs changed 1
fullSet, iter: 0  i:2, pairs changed 2
fullSet, iter: 0  i:3, pairs changed 3
fullSet, iter: 0  i:4, pairs changed 4
fullSet, iter: 0  i:5, pairs changed 5
L==H
fullSet, iter: 0  i:6, pairs changed 5
fullSet, iter: 0  i:7, pairs changed 6
fullSet, iter: 0  i:8, pairs changed 7
fullSet, iter: 0  i:9, pairs changed 8
fullSet, iter: 0  i:10, pairs changed 9
fullSet, iter: 0  i:11, pairs changed 10
L==H
fullSet, iter: 0  i:12, pairs changed 10
fullSet, iter: 0  i:13, pairs changed 11
fullSet, iter: 0  i:14, pairs changed 12
fullSet, iter: 0  i:15, pairs changed 13
fullSet, iter: 0  i:16, pairs changed 14
fullSet, iter: 0  i:17, pairs changed 15
fullSet, iter: 0  i:18, pairs changed 16
fullSet, iter: 0  i:19, pairs changed 16
fullSet, iter: 0  i:20, pairs changed 17
L==H
fullSet, iter: 0  i:21, pairs changed 17
L==H
fullSet, iter: 0  i:22, pairs changed 17
fullSet, iter: 0  i:23, pairs changed 17
L==H
fullSet

L==H
fullSet, iter: 0  i:181, pairs changed 17
fullSet, iter: 0  i:182, pairs changed 17
L==H
fullSet, iter: 0  i:183, pairs changed 17
L==H
fullSet, iter: 0  i:184, pairs changed 17
L==H
fullSet, iter: 0  i:185, pairs changed 17
L==H
fullSet, iter: 0  i:186, pairs changed 17
L==H
fullSet, iter: 0  i:187, pairs changed 17
L==H
fullSet, iter: 0  i:188, pairs changed 17
L==H
fullSet, iter: 0  i:189, pairs changed 17
L==H
fullSet, iter: 0  i:190, pairs changed 17
L==H
fullSet, iter: 0  i:191, pairs changed 17
L==H
fullSet, iter: 0  i:192, pairs changed 17
L==H
fullSet, iter: 0  i:193, pairs changed 17
L==H
fullSet, iter: 0  i:194, pairs changed 17
L==H
fullSet, iter: 0  i:195, pairs changed 17
L==H
fullSet, iter: 0  i:196, pairs changed 17
L==H
fullSet, iter: 0  i:197, pairs changed 17
L==H
fullSet, iter: 0  i:198, pairs changed 17
L==H
fullSet, iter: 0  i:199, pairs changed 17
L==H
fullSet, iter: 0  i:200, pairs changed 17
L==H
fullSet, iter: 0  i:201, pairs changed 17
L==H
fullSet, iter

L==H
fullSet, iter: 0  i:361, pairs changed 17
L==H
fullSet, iter: 0  i:362, pairs changed 17
L==H
fullSet, iter: 0  i:363, pairs changed 17
L==H
fullSet, iter: 0  i:364, pairs changed 17
L==H
fullSet, iter: 0  i:365, pairs changed 17
L==H
fullSet, iter: 0  i:366, pairs changed 17
L==H
fullSet, iter: 0  i:367, pairs changed 17
L==H
fullSet, iter: 0  i:368, pairs changed 17
L==H
fullSet, iter: 0  i:369, pairs changed 17
L==H
fullSet, iter: 0  i:370, pairs changed 17
L==H
fullSet, iter: 0  i:371, pairs changed 17
L==H
fullSet, iter: 0  i:372, pairs changed 17
L==H
fullSet, iter: 0  i:373, pairs changed 17
L==H
fullSet, iter: 0  i:374, pairs changed 17
L==H
fullSet, iter: 0  i:375, pairs changed 17
L==H
fullSet, iter: 0  i:376, pairs changed 17
L==H
fullSet, iter: 0  i:377, pairs changed 17
L==H
fullSet, iter: 0  i:378, pairs changed 17
L==H
fullSet, iter: 0  i:379, pairs changed 17
L==H
fullSet, iter: 0  i:380, pairs changed 17
L==H
fullSet, iter: 0  i:381, pairs changed 17
L==H
fullSet,

L==H
fullSet, iter: 0  i:541, pairs changed 17
L==H
fullSet, iter: 0  i:542, pairs changed 17
L==H
fullSet, iter: 0  i:543, pairs changed 17
L==H
fullSet, iter: 0  i:544, pairs changed 17
L==H
fullSet, iter: 0  i:545, pairs changed 17
L==H
fullSet, iter: 0  i:546, pairs changed 17
L==H
fullSet, iter: 0  i:547, pairs changed 17
L==H
fullSet, iter: 0  i:548, pairs changed 17
L==H
fullSet, iter: 0  i:549, pairs changed 17
L==H
fullSet, iter: 0  i:550, pairs changed 17
L==H
fullSet, iter: 0  i:551, pairs changed 17
L==H
fullSet, iter: 0  i:552, pairs changed 17
L==H
fullSet, iter: 0  i:553, pairs changed 17
L==H
fullSet, iter: 0  i:554, pairs changed 17
L==H
fullSet, iter: 0  i:555, pairs changed 17
L==H
fullSet, iter: 0  i:556, pairs changed 17
L==H
fullSet, iter: 0  i:557, pairs changed 17
L==H
fullSet, iter: 0  i:558, pairs changed 17
L==H
fullSet, iter: 0  i:559, pairs changed 17
L==H
fullSet, iter: 0  i:560, pairs changed 17
L==H
fullSet, iter: 0  i:561, pairs changed 17
L==H
fullSet,

L==H
fullSet, iter: 0  i:719, pairs changed 17
L==H
fullSet, iter: 0  i:720, pairs changed 17
L==H
fullSet, iter: 0  i:721, pairs changed 17
L==H
fullSet, iter: 0  i:722, pairs changed 17
L==H
fullSet, iter: 0  i:723, pairs changed 17
L==H
fullSet, iter: 0  i:724, pairs changed 17
L==H
fullSet, iter: 0  i:725, pairs changed 17
L==H
fullSet, iter: 0  i:726, pairs changed 17
L==H
fullSet, iter: 0  i:727, pairs changed 17
L==H
fullSet, iter: 0  i:728, pairs changed 17
L==H
fullSet, iter: 0  i:729, pairs changed 17
L==H
fullSet, iter: 0  i:730, pairs changed 17
L==H
fullSet, iter: 0  i:731, pairs changed 17
L==H
fullSet, iter: 0  i:732, pairs changed 17
L==H
fullSet, iter: 0  i:733, pairs changed 17
L==H
fullSet, iter: 0  i:734, pairs changed 17
L==H
fullSet, iter: 0  i:735, pairs changed 17
L==H
fullSet, iter: 0  i:736, pairs changed 17
L==H
fullSet, iter: 0  i:737, pairs changed 17
L==H
fullSet, iter: 0  i:738, pairs changed 17
L==H
fullSet, iter: 0  i:739, pairs changed 17
L==H
fullSet,

...

L==H
fullSet, iter: 8  i:1043, pairs changed 0
fullSet, iter: 8  i:1044, pairs changed 0
L==H
fullSet, iter: 8  i:1045, pairs changed 0
L==H
fullSet, iter: 8  i:1046, pairs changed 0
fullSet, iter: 8  i:1047, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1048, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1049, pairs changed 0
fullSet, iter: 8  i:1050, pairs changed 0
fullSet, iter: 8  i:1051, pairs changed 0
L==H
fullSet, iter: 8  i:1052, pairs changed 0
fullSet, iter: 8  i:1053, pairs changed 0
L==H
fullSet, iter: 8  i:1054, pairs changed 0
L==H
fullSet, iter: 8  i:1055, pairs changed 0
L==H
fullSet, iter: 8  i:1056, pairs changed 0
L==H
fullSet, iter: 8  i:1057, pairs changed 0
L==H
fullSet, iter: 8  i:1058, pairs changed 0
fullSet, iter: 8  i:1059, pairs changed 0
fullSet, iter: 8  i:1060, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1061, pairs changed 0
L==H
fullSet, iter: 8  i:1062, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1063, pair

j not moving enough
fullSet, iter: 8  i:1358, pairs changed 0
L==H
fullSet, iter: 8  i:1359, pairs changed 0
fullSet, iter: 8  i:1360, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1361, pairs changed 0
fullSet, iter: 8  i:1362, pairs changed 0
fullSet, iter: 8  i:1363, pairs changed 0
fullSet, iter: 8  i:1364, pairs changed 0
fullSet, iter: 8  i:1365, pairs changed 0
fullSet, iter: 8  i:1366, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1367, pairs changed 0
fullSet, iter: 8  i:1368, pairs changed 0
fullSet, iter: 8  i:1369, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1370, pairs changed 0
fullSet, iter: 8  i:1371, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1372, pairs changed 0
fullSet, iter: 8  i:1373, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1374, pairs changed 0
fullSet, iter: 8  i:1375, pairs changed 0
fullSet, iter: 8  i:1376, pairs changed 0
fullSet, iter: 8  i:1377, pairs changed 0
fullSet, iter: 8  i:1378, pairs cha

j not moving enough
fullSet, iter: 8  i:1542, pairs changed 0
fullSet, iter: 8  i:1543, pairs changed 0
fullSet, iter: 8  i:1544, pairs changed 0
fullSet, iter: 8  i:1545, pairs changed 0
fullSet, iter: 8  i:1546, pairs changed 0
fullSet, iter: 8  i:1547, pairs changed 0
fullSet, iter: 8  i:1548, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1549, pairs changed 0
fullSet, iter: 8  i:1550, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1551, pairs changed 0
fullSet, iter: 8  i:1552, pairs changed 0
fullSet, iter: 8  i:1553, pairs changed 0
fullSet, iter: 8  i:1554, pairs changed 0
fullSet, iter: 8  i:1555, pairs changed 0
fullSet, iter: 8  i:1556, pairs changed 0
fullSet, iter: 8  i:1557, pairs changed 0
fullSet, iter: 8  i:1558, pairs changed 0
fullSet, iter: 8  i:1559, pairs changed 0
L==H
fullSet, iter: 8  i:1560, pairs changed 0
fullSet, iter: 8  i:1561, pairs changed 0
fullSet, iter: 8  i:1562, pairs changed 0
fullSet, iter: 8  i:1563, pairs changed 0
fullSet, it

j not moving enough
fullSet, iter: 8  i:1720, pairs changed 0
L==H
fullSet, iter: 8  i:1721, pairs changed 0
L==H
fullSet, iter: 8  i:1722, pairs changed 0
L==H
fullSet, iter: 8  i:1723, pairs changed 0
fullSet, iter: 8  i:1724, pairs changed 0
fullSet, iter: 8  i:1725, pairs changed 0
fullSet, iter: 8  i:1726, pairs changed 0
fullSet, iter: 8  i:1727, pairs changed 0
fullSet, iter: 8  i:1728, pairs changed 0
L==H
fullSet, iter: 8  i:1729, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1730, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1731, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1732, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1733, pairs changed 0
fullSet, iter: 8  i:1734, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1735, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1736, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1737, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1738, pairs changed 0
j 

j not moving enough
fullSet, iter: 8  i:1862, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1863, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1864, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1865, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1866, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1867, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1868, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1869, pairs changed 0
fullSet, iter: 8  i:1870, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1871, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1872, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1873, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1874, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1875, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1876, pairs changed 0
j not moving enough
fullSet, iter: 8  i:1877, pairs changed 0
j not moving enough
fullSet,